In [116]:
import numpy as np
import random
import re


In [117]:
def encode_text(text, map_true):
    '''encode or decode text based on mapping passed'''
    text = text.lower()
    non_alpha_chars = re.compile('[^a-zA-Z]')
    text = non_alpha_chars.sub(" ", text)
    
    encoded_text = []
    for ch in text:
        ch = map_true.get(ch, ch) #if space then ignore
        encoded_text.append(ch)
        
    return "".join(encoded_text)

Will be using `2-grams and unigrams` to build markov model
`p(book)` is probability of a word "book" which can be broken as follows: (chain rule)
* `p(book)` reduces to `p(k/boo) * p(o/bo) * p(o/b) * p(b)`
* which again reduces to `p(k/o) * p(o/o) * p(o/b) * p(b)` due to markov assumption


Markov prob matrix will be `26 x 26`.


In [118]:
# initializing by 1 not zero to avoid zero probability for words that didnt appear in train text
bigram_mat = np.ones((26, 26))

unigram_mat = np.ones(26)

def get_index(ch):
    return ord(ch) - ord('a')

In [119]:
def update_bigram_mat(ch1, ch2):
    bigram_mat[get_index(ch1), get_index(ch2)] += 1


def update_unigram_mat(ch):
    unigram_mat[get_index(ch)] += 1

def update_prob_mats_with_sents(words):
    words = words.split()
    for word in words:
        update_unigram_mat(word[0])
        for i, ch in enumerate(word[: - 1]):
            update_bigram_mat(ch, word[i + 1])

In [120]:
#using log scale prob to counter very low values of probabilities because of very large sample space
def get_word_prob(word):
    '''returns p(word) via chain rule in log scale'''
    logp = 0
    #adding unigram prob for first letter
    logp += np.log(unigram_mat[get_index(word[0])])
    
    #bigram probs, from second letter
    for i, ch in enumerate(word[: -1]):
        logp += np.log(bigram_mat[get_index(ch), get_index(word[i + 1])])
    
    return logp
    

In [121]:
def get_sent_prob(words):
    '''returns log prob of sequence of words'''
    if type(words) == str:
        words = words.split()
    
    logp = 0
    for word in words:
        logp += get_word_prob(word)
    return logp

In [122]:
# build prob matrixs for train text
def make_prob_mat(file_name):
    global unigram_mat, bigram_mat
    not_alpha_re = re.compile('[^a-zA-Z]')
    with open(file_name, 'r', encoding='utf-8') as train:
        for line in train:
            line = line.strip()
            #if non empty line
            if line:
                #replace non alpha chars
                line = not_alpha_re.sub(" ", line)
                words = line.lower()
                update_prob_mats_with_sents(words)
    # calculate prob (normalize mats)
    unigram_mat /= unigram_mat.sum()
    bigram_mat /= bigram_mat.sum(axis = 1, keepdims=True)
                

In [123]:
class Genetic_decoder:
    
    def __init__(self, population=20):
        self.population = population
        self.dna_pool = []
        for _ in range(population):
            dna = [chr(i) for i in range(ord('a'), ord('z') + 1)]
            random.shuffle(dna)
            self.dna_pool.append(dna)
    
    
    def get_population_scores(self, encoded_text):
        scores = []
        for i, dna in enumerate(self.dna_pool):
            cur_map = {chr(ord('a') + i): dna[i] for i in range(len(dna))}
            decoded_text = encode_text(encoded_text, cur_map)
            scores.append(get_sent_prob(decoded_text))
        
        return scores
    
    
    def remove_unfits(self, scores, keep=5):
        #sort dna_pool based of scores
        indexs = range(len(scores))
        indexs = sorted(indexs, key=scores.__getitem__, reverse=True)
        self.dna_pool = [self.dna_pool[i] for i in indexs]
        self.dna_pool = self.dna_pool[: keep]
        
    
    def decipher(self, encoded_text, num_iters = 1000):
        scores = self.get_population_scores(encoded_text)
        self.remove_unfits(scores) # keep 5
        for i in range(num_iters):
            
            #make 3 child each so total will remain 20
            self.next_generation(num_children=3)
            scores = self.get_population_scores(encoded_text)
            self.remove_unfits(scores)
            if i % 100 == 0:
                print(f"Progress: {i}/{num_iters}, Best score: {max(scores)}")
                
        return {chr(ord('a') + i): self.dna_pool[0][i] for i in range(26)}
            
    
    def next_generation(self, num_children=3):
        children = []
        #making n childern per dna
        for dna in self.dna_pool:
            for _ in range(num_children):
                child = dna.copy()
                #swaping two genes (mappings) 
                i, j = random.randint(0, len(child) - 1), random.randint(0, len(child) - 1)
                child[i], child[j] = child[j], child[i]
                children.append(child)
                
        self.dna_pool.extend(children)
    
    
        

# Training on "The Magic of OZ"

In [124]:
make_prob_mat('./the_magic_of_oz.txt')

# Testing

In [125]:
# cipher is mapping of letters to unknown latters like a to x, b to z ...z to c. to break the chiper mapping shold be determined.
key_latters = [chr(i) for i in range(ord('a'), ord('z') + 1)]
mapped_latters = [chr(i) for i in range(ord('a'), ord('z') + 1)]

#creating a substitution ciphar
random.shuffle(mapped_latters)
map_true = {key_latters[i]: mapped_latters[i] for i in range(26)}

In [150]:
orignal_text = '''n recent times, cryptography has turned into a battleground of some of the world's best mathematicians and computer scientists. The ability to securely store and transfer sensitive information has proved a critical factor in success in war and business.
Because governments do not wish certain entities in and out of their countries to have access to ways to receive and send hidden information that may be a threat to national interests, cryptography has been subject to various restrictions in many countries, ranging from limitations of the usage and export of software to the public dissemination of mathematical concepts that could be used to develop cryptosystems. However, the internet has allowed the spread of powerful programs and, more importantly, the underlying techniques of cryptography, so that today many of the most advanced cryptosystems and ideas are now in the public domain.'''

In [151]:
encoded_text = encode_text(orignal_text, map_true)

In [152]:
encoded_text

'h afefhk kcofy  eaxrkldatrqx qty kzahfu chkl t gtkkpfdalzhu lw ylof lw kqf jlapu y gfyk otkqfotkcecthy thu elorzkfa yecfhkcyky  kqf tgcpckx kl yfezafpx yklaf thu kathywfa yfhyckcif chwlaotkclh qty ralifu t eackcetp wtekla ch yzeefyy ch jta thu gzychfyy  gfetzyf dlifahofhky ul hlk jcyq efaktch fhkckcfy ch thu lzk lw kqfca elzhkacfy kl qtif teefyy kl jtxy kl afefcif thu yfhu qcuufh chwlaotkclh kqtk otx gf t kqaftk kl htkclhtp chkfafyky  eaxrkldatrqx qty gffh yzgsfek kl itaclzy afykacekclhy ch othx elzhkacfy  athdchd walo pcocktkclhy lw kqf zytdf thu fmrlak lw ylwkjtaf kl kqf rzgpce ucyyfochtkclh lw otkqfotkcetp elhefrky kqtk elzpu gf zyfu kl ufifplr eaxrklyxykfoy  qljfifa  kqf chkfahfk qty tppljfu kqf yraftu lw rljfawzp raldatoy thu  olaf corlakthkpx  kqf zhufapxchd kfeqhcnzfy lw eaxrkldatrqx  yl kqtk klutx othx lw kqf olyk tuithefu eaxrklyxykfoy thu cufty taf hlj ch kqf rzgpce ulotch '

#### Using first 1000 chars for decoding

In [156]:
ga_decoder = Genetic_decoder(population=25)
key_map = ga_decoder.decipher(encoded_text, num_iters=500)

Progress: 0/500, Best score: -3478.973203625758
Progress: 100/500, Best score: -2308.333206363558
Progress: 200/500, Best score: -2183.493375800646
Progress: 300/500, Best score: -2004.808902588434
Progress: 400/500, Best score: -1861.1100939995238


In [157]:
decoded_text = encode_text(encoded_text, key_map)

In [158]:
decoded_text

'n recent times  cryptography has turned into a battleground of some of the world s best mathematicians and computer scientists  the ability to securely store and transfer sensitive information has proved a critical factor in success in war and business  because governments do not wish certain entities in and out of their countries to have access to ways to receive and send hidden information that may be a threat to national interests  cryptography has been subject to various restrictions in many countries  ranging from limitations of the usage and export of software to the public dissemination of mathematical concepts that could be used to develop cryptosystems  however  the internet has allowed the spread of powerful programs and  more importantly  the underlying techniques of cryptography  so that today many of the most advanced cryptosystems and ideas are now in the public domain '